In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from typing import List, Tuple, Dict
from IPython.core.display import display, HTML
from datasets import load_metric, load_from_disk, Dataset, DatasetDict, concatenate_datasets

import logging
logger = logging.getLogger(__name__)

# 데이터 불러오기

In [2]:
datasets = load_from_disk("/opt/ml/data/train_dataset")

train = datasets['train']
val = datasets['validation']

In [3]:
from transformers import AutoModel, AutoTokenizer

model = AutoModel.from_pretrained("klue/roberta-large")
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-large")

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaModel: ['lm_head.decoder.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it f

# 데이터 살펴보기

In [4]:
example_text_1 = train['context'][0]
print(example_text_1)

example_tokens_1 = tokenizer.tokenize(example_text_1)
print(example_tokens_1)

미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국 의회의 상원이다.\n\n미국 부통령이 상원의장이 된다. 각 주당 2명의 상원의원이 선출되어 100명의 상원의원으로 구성되어 있다. 임기는 6년이며, 2년마다 50개주 중 1/3씩 상원의원을 새로 선출하여 연방에 보낸다.\n\n미국 상원은 미국 하원과는 다르게 미국 대통령을 수반으로 하는 미국 연방 행정부에 각종 동의를 하는 기관이다. 하원이 세금과 경제에 대한 권한, 대통령을 포함한 대다수의 공무원을 파면할 권한을 갖고 있는 국민을 대표하는 기관인 반면 상원은 미국의 주를 대표한다. 즉 캘리포니아주, 일리노이주 같이 주 정부와 주 의회를 대표하는 기관이다. 그로 인하여 군대의 파병, 관료의 임명에 대한 동의, 외국 조약에 대한 승인 등 신속을 요하는 권한은 모두 상원에게만 있다. 그리고 하원에 대한 견제 역할(하원의 법안을 거부할 권한 등)을 담당한다. 2년의 임기로 인하여 급진적일 수밖에 없는 하원은 지나치게 급진적인 법안을 만들기 쉽다. 대표적인 예로 건강보험 개혁 당시 하원이 미국 연방 행정부에게 퍼블릭 옵션(공공건강보험기관)의 조항이 있는 반면 상원의 경우 하원안이 지나치게 세금이 많이 든다는 이유로 퍼블릭 옵션 조항을 제외하고 비영리건강보험기관이나 보험회사가 담당하도록 한 것이다. 이 경우처럼 상원은 하원이나 내각책임제가 빠지기 쉬운 국가들의 국회처럼 걸핏하면 발생하는 의회의 비정상적인 사태를 방지하는 기관이다. 상원은 급박한 처리사항의 경우가 아니면 법안을 먼저 내는 경우가 드물고 하원이 만든 법안을 수정하여 다시 하원에 되돌려보낸다. 이러한 방식으로 단원제가 빠지기 쉬운 함정을 미리 방지하는 것이다.날짜=2017-02-05
['미국', '상의', '##원', '또는', '미국', '상원', '(', 'United', 'State', '##s', 'Se', '##n', '##ate', ')', '은', '양', '##원', '##제', '##인', '미국', '의회'

\n\n 이 껴있다. '\' 가 [UNK]

In [5]:
example_text_2 = train['context'][7]
print(example_text_2)

example_tokens_2 = tokenizer.tokenize(example_text_2)
print(example_tokens_2)

이집트로 이어지는 해안 도로는 1개의 사단만이 방어할 수 있었으며 엘 아뎀 요새와 남아있는 영국 기갑군 잔병들에 의해서만 출입이 가능했다. 이러한 임무는 남아프리카 연방군에게도 할당되었다. 제50사단은 동쪽에서 싸우는 대안으로 좌측을 선택하여 독일 기갑 사단의 편제를 통과하거나 이탈리아 기갑사단 주위로 긴 방어선을 형성하는 것을 목표로 했다. 그들 모두를 파괴하는 것은 거의 불가능했는데, 그 이유는 사단은 혼성 부대로 이루어져 있었기 때문이다. 제5이스트요크셔와 제9더럼경보병부대가 형성한 교두보를 담당하여 이탈리아 전선으로 진입하는 것이 그들의 임무였다. 혼란과 혼돈 속에서 혼성 부대는 독일군이 장악한 동쪽보다 남쪽으로 이동하여 이집트 국경인 마델레나 요새로 이동했다. text=교두보에 있었던 적은 독일군 포병이 지원하는 이탈리아 군이었다. 그들은 기습으로 매우 놀란 것 같았다. 그들이 전 사단이 그들의 전선을 곧바로 통과하는 것을 깨달았는데 밤이 되기 전이었다. 몇몇 차량은 지뢰로 파괴되고, 다른 차량들은 불에 탔지만, 우리의 모든 사단은 적은 사상자를 냈고, 공격대대도 그들의 임무를 성공적으로 마쳤다. 보병은 마차에 탔고, 이탈리아군은 참호에 그들의 무기와 장비를 남겨둔 채 도망갔다. 더럼경보병여단의 제9대대와 제6대대의 소규모 부대는 제69여단 요새 뒤에서 주둔한 이후 해안도로를 따라 후퇴하면서 이탈리아 군인들이 돌파를 알리는 것을 보았다. 독일군 포병 및 보병과 남아프리카 공화국 보호부대의 실수로 공격을 받기는 했으나 혼성 부대는 독일군을 돌파하며 포로까지 잡았다. 6월 17일에서 18일까지 이들은 비르 엘 타라타에서 재집결했다.
['이집트', '##로', '이어지', '##는', '해안', '도로는', '1', '##개', '##의', '사단', '##만', '##이', '방어', '##할', '수', '있', '##었', '##으며', '엘', '[UNK]', '요새', '##와', '남아', '##있', '##는', '영국', '기', '##갑',

엘 아뎀 요새 -> 아뎀 [UNK]

In [6]:
example_text_3 = train['context'][8]
print(example_text_3)

example_tokens_3 = tokenizer.tokenize(example_text_3)
print(example_tokens_3)

하지만 슬픈 일들이 그녀의 행복을 질시하고 있었다.\n평생 고생만 한 어머니를 백혈병으로 잃고, 얼마 후 불경기로 가가야가 망한후 매춘부로 전락, 고생하다가 병으로 죽은 가요의 아들 노소미를 입양했다. 그 후 농민운동을 하던 고우타가 가요의 묘 앞에서 체포되어 모진 고문을 당하고 몇 년 후 사상전향을 하고 출옥하였다. 더구나 제2차 세계 대전으로 큰아들 유가 필리핀에서 전사한 사건, 이에 상처받은 수양딸 하스코(田倉初子)가 가출하는 일들도 있었다. 하지만, 가장 큰 슬픔은 사람을 죽이는 전쟁에 애국주의에 근거하여, 가담한 잘못에 죄책감을 느낀 남편의 자살이었다. 그녀는 꿋꿋히 슬픔을 참으면서 작은 아들 히토시, 엄마 오싱의 간곡한 설득으로 집에 돌아온 수양딸 하스코 그리고 딸 데이와 함께 식료품가게를 꾸려나가 여러 지점들을 둘 정도가 된다. 죽은 큰 아들 유를 대신해서 장남역할을 성실하게 해오던 히토시(田倉仁)도 부자집딸 미치코와 결혼한다. 오싱은 결혼할 남자의 누나인 하쓰코가 자신을 대접하느라 바쁜데도 전혀 돕지 않을 만큼 속없고 예의도 없는 철부지 아가씨 미치코가 마음에 전혀 들지 않았지만, 아들이 원하니까 결혼을 허락한다. 이때가 패전으로 잿더미가 된 일본경제가 성장한 1950년대였다. 하지만 오싱의 우려대로 미치코는 가출까지 할 정도로 시어머니의 속을 썩인다. 한가지 다행이라면 아들 노소미와 딸 하스코가 흥미와 적성에 맞는 도공과 수예품가게주인의 길을 걸으면서 스스로 삶을 개척하고 있다는 사실이었다.
['하지만', '슬픈', '일', '##들이', '그녀', '##의', '행복', '##을', '질', '##시', '##하고', '있', '##었', '##다', '.', '[UNK]', 'n', '##평생', '고생', '##만', '한', '어머니', '##를', '백혈병', '##으로', '잃', '##고', ',', '얼마', '후', '불경기', '##로', '가가', '##야', '##가', '망한', '##후', '매춘', '##부', '##

倉 初 [UNK]

In [7]:
tokenizer(example_text_3, return_offsets_mapping=True)

{'input_ids': [0, 3696, 10645, 1507, 7285, 3821, 2079, 4202, 2069, 1586, 2067, 19521, 1513, 2359, 2062, 18, 3, 81, 28231, 6671, 2154, 1891, 4226, 2138, 19716, 6233, 1509, 2088, 16, 3943, 1943, 27571, 2200, 26573, 2275, 2116, 28924, 2158, 20877, 2144, 2200, 9364, 16, 6671, 2205, 4795, 1158, 6233, 1565, 2073, 7390, 2079, 4200, 766, 2024, 2044, 2138, 11930, 2371, 2062, 18, 636, 1943, 6290, 7796, 2069, 1889, 2414, 594, 2137, 2256, 2116, 7390, 2079, 1087, 1388, 27135, 7176, 2496, 2051, 20224, 6689, 2069, 5340, 2088, 1077, 761, 1943, 4472, 2165, 2482, 2069, 6159, 1675, 2349, 2205, 2507, 2062, 18, 7674, 1545, 2302, 2232, 3665, 4431, 6233, 1751, 4383, 5704, 8418, 27135, 8601, 2470, 3863, 16, 1504, 2170, 5456, 2757, 2073, 15651, 2905, 1889, 4891, 12, 435, 3, 3, 313, 13, 543, 17018, 2205, 2259, 1507, 2031, 2119, 1513, 2359, 2062, 18, 3696, 16, 3676, 1751, 7799, 2073, 3611, 2069, 6810, 2259, 4175, 2170, 9454, 3802, 2170, 4952, 7488, 16, 11573, 2470, 4389, 2170, 16289, 2069, 6629, 4507, 2079, 6205

input_ids와 offset mapping을 사용하여 UNK 토큰에 해당하는 한국어 토큰이 무엇인지 알아낼 수 있겠다.

# Train data _ UNK 토큰 분석

In [8]:
def find_UNK(examples : List) -> List :
    UNK_words = []  # UNK에 해당하는 단어를 담을 리스트
    UNK_counts = [] # UNK 토큰의 개수를 담을 리스트
    tokenized_examples = tokenizer(examples, return_offsets_mapping=True)
    for index in range(len(examples)) :
        res = []
        unk_idx = [idx for idx, token in enumerate(tokenized_examples['input_ids'][index]) if token == tokenizer.unk_token_id] # unk 토큰의 id에 해당하는 토큰만 뽑기
        UNK_counts.append(len(unk_idx)) # UNK 토큰 개수 세기
        for token in unk_idx :
            start, end = tokenized_examples['offset_mapping'][index][token] # offset mapping을 참고하여 원래 단어의 start, end를 찾고
            res.append(examples[index][start:end]) # 원래 단어를 추가
        UNK_words.append(res)
    return UNK_words, UNK_counts

## 우선은 question 과 answer 살펴보기

In [9]:
# answer 데이터는 별도로 뽑아내는 작업 필요
train_answer = [''.join(train['answers'][i]['text']) for i in range(len(train))]

In [10]:
# train의 question과 answer를 함수에 넣어준다.
train_question_UNK, train_question_UNK_cnt = find_UNK(train['question'])
train_answer_UNK, train_answer_UNK_cnt = find_UNK(train_answer)

In [11]:
# question과 answer에도 UNK 토큰이 있었다니 ...
print(f'Question에는 총 {sum(train_question_UNK_cnt)} 개')
print(f'Answer에는   총 {sum(train_answer_UNK_cnt)} 개')

Question에는 총 64 개
Answer에는   총 95 개


In [12]:
# 어떤 단어가 UNK 토큰으로 치환되었는지 살펴보자.
def find_original_word(UNK_tokens: List[List]) -> List:
    orig_word = []
    for example in UNK_tokens :
        if example :
            orig_word.extend(example)
    
    return orig_word

In [13]:
train_qst_orig = find_original_word(train_question_UNK)
train_ans_orig = find_original_word(train_answer_UNK)

In [14]:
print(train_qst_orig)
print('-------------------------------------------------------------------------------------------------------------------------------')
print(train_ans_orig)

['홀뮴이', '옻칠의', '쾰른토나토아', '프롱트낵', '배곯는', '뮈라가', '폴리펩타이드', '놋쇠로', '수퇘지의', '멎게', '도스토옙스키의', '젱킨스의', '여읜', '아델리펭귄', '도스토옙스키는', '도스토옙스키의', '리위안훙이', '트롱트낵', '쫒을', '떄', '스콴토가', '윅', '굿펠로우나무타기캥거루가', '스뮈츠가', '아델리펭귄의', '줏키의', '셧다운을', '뮈소가', '썻던', '도롱뇽과', '촐라푸람을', '수탉은', '테르모퓔라이', '함꼐', 'حصروم', '하탸야', '비숑', '스퀠쾨더가', '퍄샤는', '보괸되어', '派', '핼시를', '번쨰', '댜틀로프가', '펨토의', '옻칠의', '홋카이도제국대학의', '뮈라가', '니콜라옙스크', '쿠첵에게', '이셴', '댜틀로프를', '쥘레의', '시오야슌이', '벵골만', '아퀼라를', '윔블던과', '쿠첵과의', '어댑터로', '갬블러', '뮈소의', '뫼비우스가', '굿펠로우나무타기캥거루가', '헵번과']
-------------------------------------------------------------------------------------------------------------------------------
['優', '塡', '벵골', '識', '햣켄', '忠', '괭이갈매기', '魯', '陵', '塔', '쾰른', '片', '刀', '도스토옙스키가', '아버스놋', '陳', '毅', '釋', '總', '∧', '맬컴', '勳', 'Gräbner', '宋', '祠', '니촐라', '夔', '弼', '蓮', '塔', 'のようにならないようにしよう', '割', '卵', '퓌스콘', '麻', '辣', '汪', '탯줄', '循', '格', '웡원하오', '坂', '駅', '鉱', '겜린', '솔레놉신', '蔣', '儼', '베들레헴의', '流', '遷', '奎', '벵츠도터', '媒', '槪', '念', '扶', '餘', '縣', '監'

## context 데이터 살펴보기

In [15]:
# context도 똑같이 어떤 단어가 UNK 토큰으로 치환되었는지 살펴보자.
train_context_UNK, train_context_UNK_cnt = find_UNK(train['context'])

Token indices sequence length is longer than the specified maximum sequence length for this model (1022 > 512). Running this sequence through the model will result in indexing errors


In [16]:
# 우선 300개만 뽑아보자 -> 역시 '\' 가 많다.
train_ctx_orig = find_original_word(train_context_UNK)
print(train_ctx_orig[:300])

['\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '諭', '龕', '\\', '\\', '\\', '\\', '背', '冠', '\\', '\\', '冠', '函', '\\', '\\', '癸', '巳', '施', '承', '兩', '\\', '\\', '\\', '\\', '\\', '\\', '퓌헤레노르', '\\', '\\', '館', '驛', '횃불을', '更', '\\', '\\', '\\', '\\', '樊', '아뎀', '\\', '倉', '初', '倉', '后', '\\', '\\', '\\', '\\', '蘿', '蔔', '護', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '李', '淳', '鎔', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '草', '\\', '\\', '베르됭에서', '베르됭', '베르됭은', '\\', '\\', '\\', '\\', '베르됭에서의', '\\', '\\', '\\', '\\', '깰', '\\', '\\', '\\', '\\', '梅', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '쪼갰고', '恒', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\',

In [17]:
# \를 (정확히 말하면 \n) 제외한 UNK 토큰과 그 개수를 알아보자.
def pick_no_slash(UNK_tokens : List[List]) :
    UNK_no_slash = [] # '\\'가 아닌 UNK 토큰만 담을 리스트
    UNK_per_data = [] # '\\'를 제외한 데이터 하나 당 UNK 토큰 개수

    for tokens in UNK_tokens :
        cnt = 0
        for token in tokens :
            if token != '\\' and token != ' ' :
                cnt += 1
                UNK_no_slash.append(token)
        UNK_per_data.append(cnt)
        
    return UNK_no_slash, UNK_per_data

In [18]:
train_ctx_no_slash, train_UNK_per_context = pick_no_slash(train_context_UNK)

print(train_ctx_no_slash[:100])
print(train_UNK_per_context[:100])

['諭', '龕', '背', '冠', '冠', '函', '癸', '巳', '施', '承', '兩', '퓌헤레노르', '館', '驛', '횃불을', '更', '樊', '아뎀', '倉', '初', '倉', '后', '蘿', '蔔', '護', '李', '淳', '鎔', '草', '베르됭에서', '베르됭', '베르됭은', '베르됭에서의', '깰', '梅', '쪼갰고', '恒', '羽', '變', '態', '胸', '腺', '器', '幼', '若', '蟲', 'Πάτρα', 'Πάτραι', '素', '髮', '優', '塡', '李', '昌', '쿨부샨', '다퉜지만', '챕터에서', '狂', '笑', '왐파노아그', '遇', '巖', '沙', '落', '雁', '홀뮴은', '홀뮴은', 'n홀뮴', '어븀', '홀뮴의', '유로퓸', '터븀의', 'n홀뮴은', '홀뮴은', '聯', '邦', '衆', '国', '鼻', '音', '아비뇽의', 'ملك', 'المملوك', 'mulūk', '핼리팩스로', '쭌이', '없앴다는', '코눙그', '–', '헤르츨의', '⇌', '奏', '儒', '達', '梅', '乃', '妙', '벵골', '벵골', '벵골']
[0, 0, 1, 4, 6, 1, 5, 1, 3, 4, 0, 0, 0, 3, 0, 1, 4, 1, 1, 0, 2, 0, 0, 0, 9, 0, 2, 0, 0, 0, 4, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 3, 0, 0, 1, 0, 5, 0, 0, 0, 0, 0, 9, 0, 0, 0, 4, 0, 0, 0, 0, 3, 3, 0, 0, 0, 1, 0, 1, 3, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 2, 0, 3, 0, 0, 7, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0]


In [19]:
# UNK 개수를 모두 세어보자.
print(f'train context의 UNK 토큰은 총 {len(train_ctx_orig)} 개 이다.')
print(f'전체 train context에서 1개 context 마다 평균적으로 몇 개의 UNK 토큰이 있을까 ? (\\\ 포함)      {len(train_ctx_orig) / len(train)} 개')
print(f'UNK 토큰이 하나도 없는 context를 제외하고, UNK 토큰이 있는 context의 평균 UNK 토큰 개수는?    {len(train_ctx_orig) / (len(train) - train_UNK_per_context.count(0))} 개')

train context의 UNK 토큰은 총 28279 개 이다.
전체 train context에서 1개 context 마다 평균적으로 몇 개의 UNK 토큰이 있을까 ? (\\ 포함)      7.155617408906883 개
UNK 토큰이 하나도 없는 context를 제외하고, UNK 토큰이 있는 context의 평균 UNK 토큰 개수는?    18.291720569210867 개


In [20]:
train_ctx_UNK_uniq = list(set(train_ctx_orig))

In [21]:
print(f'train context의 UNK 토큰 개수는 \\\를 제외하고 {sum(train_UNK_per_context)} 개 이다.')
print(f'train context의 모든 중복을 제거한 UNK 토큰은  {len(train_ctx_UNK_uniq)} 개 이다.')

train context의 UNK 토큰 개수는 \\를 제외하고 5471 개 이다.
train context의 모든 중복을 제거한 UNK 토큰은  2491 개 이다.


## 주목해야할 점 !
    - \\n을 없애면 context의 UNK 토큰 총 28279개를 5471개로 줄일 수 있다. -> 일반 토큰으로 바꿔도 될까 ?
    - 5471개 중에서도 중복을 제외하면 2490개 가 남는다.
    - question과 answer에 있는 UNK 토큰을 우선 없애야 한다.
    - 베르됭에서, 베르됭은, 베르됭에서의 ... 섄탈의, 섄탈은, 섄탈을, 섄탈에, ... 등 과 같이 명사 하나를 토큰으로 추가해주면 없앨 수 있는 UNK 토큰들도 많다.
    - 떄, 맟추어, 푝력을 등과 같이 오탈자로 인해 인식하지 못하는 토큰도 있다.

# 새로운 토큰 추가하기

## 1. question

In [22]:
word_to_erase = '은 는 이 가 로 게 의 과 을 를 던 에게 \\ 으로 으로써 처럼 으로 아 이라 지만 \
                 에서 먹어 하고 으로만 으면 로부터 하여 다 으라고 까지 이나 고 와 으며 이었다 \
                 으로서 보다 인 니다 이다 였다 다 거리듯 거나 에서의 거릴 에 어 였다고 다는 들을 \
                 들과 들의 어져 네까 네 으로는 으나 다고 부터 해주는 아서 에서는 아터진 숴 한 로만 만을 는가 \
                 와는 워준 다가 워 있는 들 위치 위치에서 위치로 위치를 위치에 주변에 도 으리라 으리라고 \
                 들은 버렸다 버리고 로서 에는 었는데 된 하게 았다 떨어졌다가 되며 라는 니까 장면 명 과의'.split()

word_to_erase = sorted(word_to_erase, reverse=True, key=len)
print(word_to_erase)

['떨어졌다가', '위치에서', '으리라고', '으로써', '으로만', '로부터', '으라고', '이었다', '으로서', '거리듯', '에서의', '였다고', '으로는', '해주는', '에서는', '아터진', '위치로', '위치를', '위치에', '주변에', '으리라', '버렸다', '버리고', '었는데', '에게', '으로', '처럼', '으로', '이라', '지만', '에서', '먹어', '하고', '으면', '하여', '까지', '이나', '으며', '보다', '니다', '이다', '였다', '거나', '거릴', '다는', '들을', '들과', '들의', '어져', '네까', '으나', '다고', '부터', '아서', '로만', '만을', '는가', '와는', '워준', '다가', '있는', '위치', '들은', '로서', '에는', '하게', '았다', '되며', '라는', '니까', '장면', '과의', '은', '는', '이', '가', '로', '게', '의', '과', '을', '를', '던', '\\', '아', '다', '고', '와', '인', '다', '에', '어', '네', '숴', '한', '워', '들', '도', '된', '명']


In [23]:
# 토큰의 조사를 제거하고 new_tokens 리스트에 추가한다.
def add_new_tokens(token_list: List) :
    new_tokens = []
    for token in token_list :
        for proposition in word_to_erase :
            if token[-len(proposition):] == proposition :
                trans_table = token.maketrans(proposition, ' ' * len(proposition))
                new_token = token[:-len(proposition)] + token[-len(proposition)].translate(trans_table)
                new_tokens.append(new_token.rstrip())
                break
        else :
            new_tokens.append(token)
    return new_tokens

In [24]:
# 원래 UNK 토큰으로 인식되었던 단어들 _ question
print(train_qst_orig)

['홀뮴이', '옻칠의', '쾰른토나토아', '프롱트낵', '배곯는', '뮈라가', '폴리펩타이드', '놋쇠로', '수퇘지의', '멎게', '도스토옙스키의', '젱킨스의', '여읜', '아델리펭귄', '도스토옙스키는', '도스토옙스키의', '리위안훙이', '트롱트낵', '쫒을', '떄', '스콴토가', '윅', '굿펠로우나무타기캥거루가', '스뮈츠가', '아델리펭귄의', '줏키의', '셧다운을', '뮈소가', '썻던', '도롱뇽과', '촐라푸람을', '수탉은', '테르모퓔라이', '함꼐', 'حصروم', '하탸야', '비숑', '스퀠쾨더가', '퍄샤는', '보괸되어', '派', '핼시를', '번쨰', '댜틀로프가', '펨토의', '옻칠의', '홋카이도제국대학의', '뮈라가', '니콜라옙스크', '쿠첵에게', '이셴', '댜틀로프를', '쥘레의', '시오야슌이', '벵골만', '아퀼라를', '윔블던과', '쿠첵과의', '어댑터로', '갬블러', '뮈소의', '뫼비우스가', '굿펠로우나무타기캥거루가', '헵번과']


In [25]:
# 조사를 지운, 새로 추가할 토큰을 new_tokens 리스트에 한꺼번에 담아놓자.
new_tokens = add_new_tokens(train_qst_orig)
print(new_tokens)

['홀뮴', '옻칠', '쾰른토나토', '프롱트낵', '배곯', '뮈라', '폴리펩타이드', '놋쇠', '수퇘지', '멎', '도스토옙스키', '젱킨스', '여읜', '아델리펭귄', '도스토옙스키', '도스토옙스키', '리위안훙', '트롱트낵', '쫒', '떄', '스콴토', '윅', '굿펠로우나무타기캥거루', '스뮈츠', '아델리펭귄', '줏키', '셧다운', '뮈소', '썻', '도롱뇽', '촐라푸람', '수탉', '테르모퓔라', '함꼐', 'حصروم', '하탸야', '비숑', '스퀠쾨더', '퍄샤', '보괸되', '派', '핼시', '번쨰', '댜틀로프', '펨토', '옻칠', '홋카이도제국대학', '뮈라', '니콜라옙스크', '쿠첵', '이셴', '댜틀로프', '쥘레', '시오야슌', '벵골만', '아퀼라', '윔블던', '쿠첵', '어댑터', '갬블러', '뮈소', '뫼비우스', '굿펠로우나무타기캥거루', '헵번']


## 2. answer

In [26]:
# 원래 UNK 토큰으로 인식되었던 단어들 _ answer
print(train_ans_orig)

['優', '塡', '벵골', '識', '햣켄', '忠', '괭이갈매기', '魯', '陵', '塔', '쾰른', '片', '刀', '도스토옙스키가', '아버스놋', '陳', '毅', '釋', '總', '∧', '맬컴', '勳', 'Gräbner', '宋', '祠', '니촐라', '夔', '弼', '蓮', '塔', 'のようにならないようにしよう', '割', '卵', '퓌스콘', '麻', '辣', '汪', '탯줄', '循', '格', '웡원하오', '坂', '駅', '鉱', '겜린', '솔레놉신', '蔣', '儼', '베들레헴의', '流', '遷', '奎', '벵츠도터', '媒', '槪', '念', '扶', '餘', '縣', '監', '劉', '肥', '冬', '牲', '池', '琿', '郵', '왝시', '祭', '壇', '에스파뇰', '康', '藏', '臺', '°', '趙', '匡', '오툉의', '興', '輪', '쾰른', '횃불', '狼', '孟', '귈리엘모', '甚', '次', '퓌스텔', '다롄', '뮤코펩티드', '로엡이', '呂', '超', '淵', '祚']


In [27]:
# 조사를 지운, 새로 추가할 토큰들
print(add_new_tokens(train_ans_orig))

['優', '塡', '벵골', '識', '햣켄', '忠', '괭이갈매기', '魯', '陵', '塔', '쾰른', '片', '刀', '도스토옙스키', '아버스놋', '陳', '毅', '釋', '總', '∧', '맬컴', '勳', 'Gräbner', '宋', '祠', '니촐라', '夔', '弼', '蓮', '塔', 'のようにならないようにしよう', '割', '卵', '퓌스콘', '麻', '辣', '汪', '탯줄', '循', '格', '웡원하오', '坂', '駅', '鉱', '겜린', '솔레놉신', '蔣', '儼', '베들레헴', '流', '遷', '奎', '벵츠도터', '媒', '槪', '念', '扶', '餘', '縣', '監', '劉', '肥', '冬', '牲', '池', '琿', '郵', '왝시', '祭', '壇', '에스파뇰', '康', '藏', '臺', '°', '趙', '匡', '오툉', '興', '輪', '쾰른', '횃불', '狼', '孟', '귈리엘모', '甚', '次', '퓌스텔', '다롄', '뮤코펩티드', '로엡', '呂', '超', '淵', '祚']


In [28]:
# new_tokens 리스트에 한꺼번에 담아놓자.
new_tokens.extend(add_new_tokens(train_ans_orig))
print(new_tokens)

['홀뮴', '옻칠', '쾰른토나토', '프롱트낵', '배곯', '뮈라', '폴리펩타이드', '놋쇠', '수퇘지', '멎', '도스토옙스키', '젱킨스', '여읜', '아델리펭귄', '도스토옙스키', '도스토옙스키', '리위안훙', '트롱트낵', '쫒', '떄', '스콴토', '윅', '굿펠로우나무타기캥거루', '스뮈츠', '아델리펭귄', '줏키', '셧다운', '뮈소', '썻', '도롱뇽', '촐라푸람', '수탉', '테르모퓔라', '함꼐', 'حصروم', '하탸야', '비숑', '스퀠쾨더', '퍄샤', '보괸되', '派', '핼시', '번쨰', '댜틀로프', '펨토', '옻칠', '홋카이도제국대학', '뮈라', '니콜라옙스크', '쿠첵', '이셴', '댜틀로프', '쥘레', '시오야슌', '벵골만', '아퀼라', '윔블던', '쿠첵', '어댑터', '갬블러', '뮈소', '뫼비우스', '굿펠로우나무타기캥거루', '헵번', '優', '塡', '벵골', '識', '햣켄', '忠', '괭이갈매기', '魯', '陵', '塔', '쾰른', '片', '刀', '도스토옙스키', '아버스놋', '陳', '毅', '釋', '總', '∧', '맬컴', '勳', 'Gräbner', '宋', '祠', '니촐라', '夔', '弼', '蓮', '塔', 'のようにならないようにしよう', '割', '卵', '퓌스콘', '麻', '辣', '汪', '탯줄', '循', '格', '웡원하오', '坂', '駅', '鉱', '겜린', '솔레놉신', '蔣', '儼', '베들레헴', '流', '遷', '奎', '벵츠도터', '媒', '槪', '念', '扶', '餘', '縣', '監', '劉', '肥', '冬', '牲', '池', '琿', '郵', '왝시', '祭', '壇', '에스파뇰', '康', '藏', '臺', '°', '趙', '匡', '오툉', '興', '輪', '쾰른', '횃불', '狼', '孟', '귈리엘모', '甚', '次', '퓌스텔', '다롄', '뮤코펩티드', '로엡', '呂', 

## 3. context

In [29]:
print(train_ctx_orig[:300])

['\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '諭', '龕', '\\', '\\', '\\', '\\', '背', '冠', '\\', '\\', '冠', '函', '\\', '\\', '癸', '巳', '施', '承', '兩', '\\', '\\', '\\', '\\', '\\', '\\', '퓌헤레노르', '\\', '\\', '館', '驛', '횃불을', '更', '\\', '\\', '\\', '\\', '樊', '아뎀', '\\', '倉', '初', '倉', '后', '\\', '\\', '\\', '\\', '蘿', '蔔', '護', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '李', '淳', '鎔', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '草', '\\', '\\', '베르됭에서', '베르됭', '베르됭은', '\\', '\\', '\\', '\\', '베르됭에서의', '\\', '\\', '\\', '\\', '깰', '\\', '\\', '\\', '\\', '梅', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '쪼갰고', '恒', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\', '\\',

In [30]:
# 확인해보니 \가 ''으로 잘 치환되었음을 알 수 있다. -> '\n'은 직접 추가해주도록 하자. (없)앴다. 보살핌 슬펐 올림퓌아 톳 급증 ← 숫자 지우기, ° 추가 n 빼기
print(add_new_tokens(train_ctx_orig[:500])) 

['', '', '', '', '', '', '', '', '', '', '', '諭', '龕', '', '', '', '', '背', '冠', '', '', '冠', '函', '', '', '癸', '巳', '施', '承', '兩', '', '', '', '', '', '', '퓌헤레노르', '', '', '館', '驛', '횃불', '更', '', '', '', '', '樊', '아뎀', '', '倉', '初', '倉', '后', '', '', '', '', '蘿', '蔔', '護', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '李', '淳', '鎔', '', '', '', '', '', '', '', '', '', '', '', '', '', '草', '', '', '베르됭', '베르됭', '베르됭', '', '', '', '', '베르됭', '', '', '', '', '깰', '', '', '', '', '梅', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '쪼갰', '恒', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '羽', '變', '態', '', '', '胸', '腺', '器', '', '', '幼', '若', '蟲', 'Πάτρα', 'Πάτραι', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',

In [31]:
new_tokens.extend(add_new_tokens(train_ctx_orig))

# 중복 제거
new_tokens = list(set(new_tokens))

In [32]:
new_tokens[new_tokens.index('올륌피')] = '올륌퓌아'
new_tokens.remove('톳유부영양밥')
new_tokens.remove('톳멸치볶음')
new_tokens.remove('톳밥')
new_tokens[new_tokens.index('급증시킵')] = '시킵'
new_tokens[new_tokens.index('←카르보니움')] = '카르보니움'
new_tokens[new_tokens.index('←단소')] = '←'

In [33]:
new_tokens[new_tokens.index('200～500명')] = '～'

In [34]:
new_tokens.remove('')
new_tokens.remove('35°')
new_tokens.remove('298°')
new_tokens.remove('1°')
new_tokens.remove('180°')
new_tokens.remove('40°까지')
new_tokens.remove('45°이하')
new_tokens.remove('270°')
new_tokens.remove('90°')

In [35]:
new_tokens[0] = '\\n'

In [36]:
for idx in range(len(new_tokens)) :
    if new_tokens[idx][0] == 'n' :
        new_tokens[idx] = new_tokens[idx][1:]

In [37]:
tokenizer.tokenize('나쁩니까?"')

['[UNK]', '?', '"']

In [38]:
for i in train['context'] :
    if '나쁩니까' in i :
        print(i)

1839년 7월에 유진길은 집에서 체포되었다. 그의 친척들 중 많은 사람들이 그에게 배교를 간청했지만, 그는 그러기를 거절했다. 그들은 그에게 그의 가족들이게 벌어질 일과 처지 그리고 재산의 손실을 일러주었지만, 그는 그들에게 그들에게 문제를 안긴 것은 미안하지만, 몸을 돌보는 것 보다는 영혼을 구원받는 것이 더 중요하다고 말했다.\n\n포장은 그를 심문했다. "너는 관리로서 어떻게 조정에서 금하는 종교를 고수할 수 있느냐? 천주교 교인들과 서적들이 어디에 있는지 대거라." 유진길은 아무것도 누설하지 않았고, 따라서 다섯 차례 격심한 고문을 받아 그의 살가죽이 뜯겨 나갔다.\n\n포장은 유진길에게 앵베르 주교와 두 명의 다른 선교사들에 대해서 물었다. 유진길은 그에게 그들은 조선의 백성들에게 하느님을 가르치고 그들의 영혼의 구원을 돕고자 조선에 왔다고 말했다. 그는 그 선교사들은 결코 자신들의 부와 명예 그리고 쾌락을 추구하지 않는다고 말했다. 포장은 그들을 조선으로 보낸 사람이 누구냐고 물었다. 유진길은 자신이 그랬다고 말했다. 그러자 포장은 앵베르 주교를 데러와서 그들에게 같이 물었다. 주교는 유진길에게 조정은 이미 모방 신부와 샤스탕 신부가 조선에 있는 것을 알고 있다고 말했다.\n\n그러나, 유진길은 조선에 있는 교회 지도자들의 이름을 대기를 거부했다. 그의 다리는 주뢰형으로 뒤틀려 버렸고, 밧줄에 묶인 채 엄청난 양의 피를 흘리고 있었다. 포장은 심문을 계속했다. "이것은 너 같이 우둔하고 낮은 신분의 사람이 혼자서 저지를 수 있는 죄가 아니다. 천주교도들 중 누가 이 일을 지시했느냐? 네가 네 나라의 아름다운 전통과 의식을 버리고 외국인들의 불충한 풍습을 받아들였기 때문에, 네가 만 번을 사형에 처해진다 하여도, 그 형벌은 너무 가볍지 않겠느냐? 이것은 엄숙한 심문이다. 따라서 한 치의 거짓도 없이 신중히 답하거라." 그들은 천주교의 가르침이 그릇되며 불충하고 반사회적이기 때문에 외국인 신부들을 조선으로 보낸 사람들은 반역죄를 지었음을 강조했다.\n

In [39]:
print(new_tokens)

['\\n', 'völva', '韜', '貢', '顧', '嶽', '（', '웡원하오', '淳', 'حزب', '儒', '紂', '쫒', 'ὁπλίτης', '～', '喬', '鳥', '샥티파', '兀', '叉', '洲', '위스트레헴', '캇츠', '佩', '碑', '逆', '危', '议', '壽', '햐쿠하치', '贛', '賀', '薄', '핼시', 'à', '켤', '뗏목', '素', '虞', '爺', 'Fesztiválzenekar', '堀', '旣', '延', '谷', '胡', '압뒬카디르', '流', '沙', '鍾', '웸블리', '派', '퀀틱', 'ク', '꿰맸', '壬', '떤선녓', '刹', '塚', '畸', '索', '誤', '호엔촐레른', '球', 'Пак', '詠', 'Λιμός', '뤽상부르궁', '지샜', '架', '誌', '紋', '継', '땝납', '識', '욧카이치', '庚', '槪', '팼', '送', 'Moái', '퀘벡ᐨ파리문학상', '蠻', '誹', '叭', '趺', '彝', '龕', '層', 'E°', '妙', '고르노바다흐샨', '삥삥', '번쨰', '牟', '樞', '鏞', '홑몸', '饗', '浪', '隠', '힉먼', '耆', '否', 'امام', '윅', '排', 'Cúscraid', '蓋', 'Elín', '紹', 'சாம்ராஜ்யம்', 'Ｖ', '学', '結', 'アッカーマン', '잽', '普', '듄', 'thiên', '癲', 'Chèvremont', '庵', '偃', '꾸밈', '히믈러', '航', '禑', '횃불', '幹', '３', '槍', '亥', '閭', '廳', '清', '隋', '館', '符', '务', '視', '矢', '優', '韻', '免', '돛대', '揮', '옝기사르', '輪', '炎', '띌', '薛', '어댑터', '律', '納', '캇페', '奴', '샥터', '←', '起', '휜', '넙다리뼈', '壹', '±', '他', '뮐베르크', '香', 'mulūk', '먀

### 오타도 포함시켜야할까 ?

new_tokens를 정리하고 나서 정리된 토큰들이 vocab에 있는지도 확인 

## test

In [38]:
new_tokens = ['völva', '韜', '貢', '顧', '嶽', '（', '웡원하오', '淳', 'حزب', '儒', '紂', '쫒', 'ὁπλίτης', '～', '喬', '鳥', '샥티파', '兀', '叉', '洲', '위스트레헴', '캇츠', '佩', '碑', '逆', '危', '议', '壽', '햐쿠하치', '贛', '賀', '薄', '핼시', 'à', '켤', '뗏목', '素', '虞', '爺', 'Fesztiválzenekar', '堀', '旣', '延', '谷', '胡', '압뒬카디르', '流', '沙', '鍾', '웸블리', '派', '퀀틱', 'ク', '꿰맸', '壬', '떤선녓', '刹', '塚', '畸', '索', '誤', '호엔촐레른', '球', 'Пак', '詠', 'Λιμός', '뤽상부르궁', '지샜', '架', '誌', '紋', '継', '땝납', '識', '욧카이치', '庚', '槪', '팼', '送', 'Moái', '퀘벡ᐨ파리문학상', '蠻', '誹', '叭', '趺', '彝', '龕', '層', 'E°', '妙', '고르노바다흐샨', '삥삥', '번쨰', '牟', '樞', '鏞', '홑몸', '饗', '浪', '隠', '힉먼', '耆', '否', 'امام', '윅', '排', 'Cúscraid', '蓋', 'Elín', '紹', 'சாம்ராஜ்யம்', 'Ｖ', '学', '結', 'アッカーマン', '잽', '普', '듄', 'thiên', '癲', 'Chèvremont', '庵', '偃', '꾸밈', '히믈러', '航', '禑', '횃불', '幹', '３', '槍', '亥', '閭', '廳', '清', '隋', '館', '符', '务', '視', '矢', '優', '韻', '免', '돛대', '揮', '옝기사르', '輪', '炎', '띌', '薛', '어댑터', '律', '納', '캇페', '奴', '샥터', '←', '起', '휜', '넙다리뼈', '壹', '±', '他', '뮐베르크', '香', 'mulūk', '먀오디거우', '臂', '謀', '癎', 'Φάλαγγα', '星', '疾', '泛', '황즈슝', '專', '腱', 'lǫgmaðr', '베렝가르', '止', '恋', '浩', '제라프샨강', 'み', '郷', '恕', '杖', '노보셀렝긴스크', '羅', '除', '아델리펭귄', '뷜로', 'に', '役', 'wēnlŭmiàn', '羨', '墻', 'Thiêm', '貊', '旻', '켁', '買', '쵀디', '샨시성', '途', '창립멥버', 'ㅎ뢀약', '渲', 'ˈjazik', '参', '부디놋', '丈', 'し', '競', '伺', '歲', '旗', '網', '街', '貞', '벵골만', '週', '테셴', '닻', '池', '救', '舞', '嬪', '粒', '灝', '次', '六', '巡', '黎', '카탸', '塽', '雁', 'đường', '域', '印', '承', '퓌르논', '淞', '肥', '殻', '璽', '敬', '執', '糞', '쾰른', '贊', '킵초케', '갬블', '猛', '雙', '呂', '味', '羌', '磁', 'minīn', '큭큭', '堵', '談', '葉', '特', '湜', '寶', '鬼', '瑞', '直', '夏', '順', '색옻칠', '壇', '禅', '쇤스키', '品', '耽', '件', '健', '毘', '柄', '青', '老', '묽', '戦', '뷀츠부르크', '構', '左', '慾', '야르믜슈', '惣', '쾰른', '霊', '調', '프리츨라어', '챕터나', '營', '슬펐', '留', '야쿱', '劑', '厲', '폽풀라레스', '犯', '恂', '醫', '仙', '歷', '여읜', '腹', '맟추', '어븀', '鑊', '鹽', '聞', '急', '省', '狂', 'ならはしみき', '析', '湯', '症', '點', '就', 'Παφλαγών', '옻칠기', '首', 'François', '竜', 'Amīr', '增', '소꿉친구', '潘', '瓊', '雪', '糖', '嬴', '奏', '도롱뇽목', '衒', '娠', 'დავით', '쓰촨', '罪', '쿠첵', '瀛', '쫒아내', '게임스폿', '髻', '柿', '라오읻싸', '唐', 'Πόντος', '篤', '炉', '邦', '와치랄롱꼰', '煥', '아퀼레이', '蔔', '襄', '銅', 'moˈnar', '윔블던', '蒿', '켐페', '樊', '나눗셈값', '아녜스', '淖', '１５세', '畵', '賈', '鎬', '積', '묄렌도르프', 'のマニフェスト', '兩', '那', '料', '포르텡베생', '緩', '賊', '緣', '폴랸스키', '勉', 'Πάτραι', '堤', '모켕탕동', '４분', '脚', '顫', '伊', '話', '囚', '도룡뇽', '閑', '炳', 'さよなら', '유로퓸', '功', '染', '杉', '티모셴코', '栗', '圈', '易', '櫓', '丁', '散', '婦', '위웡섬', '퓔레', '맬콤', 'pāiza', '라오읻싸라', '峻', '笑', '恒', '点', '４킬로', '来', 'インカツ', '뮐', '番', '촐라', '落', '準', '브륀힐드', '남캅카스', '翊', '廻', '会', '뗀', '晉', '勤', '孤', '寫', '塡', 'Những', '折', '曜', '스퀠쾨더', '曆', '閃', '땔감', '剛', '災', 'ə', 'Jơi', '某', '尉', '簒', '連', '나뮈르', '촐몬데레', '琶', '무깟디마', '倩', '틔웠', '핥', '이스파냑', '肅', '°', '보살핌', '混', '膏', '寧', '践', '시퀀스', '吾', '브런즈윅', '央', '뭍', '쿨부샨', '揭', 'ＭＳ대', '聾', '햅번', '片', '菌', '浸', '若', '失', '幼', '캇파투카', '授', '테르모퓔라', '촐라국', '셍캉', '뫼즈', '涿', '眉', '跏', '影', '塔', '刑', '循', '黑', '암탉', '組', '비숑', '引', '靈', '寄', '鎔', '캥거루섬', '念', 'حصروم', '宴', '윔베르', '쑹화', '翔', '스텟', '籽', '嫡', 'إمام', '쓰촨성', '獻', '謙', '겻', '裕', '質', '浜', '鎌', '威', '梵', '桓', '이리듐', '遊', '接', '射', '裹', '康', '齒', '굄목', '쿰셍기르', '鴻', '廉', '円', '狗', '메나헴', '髮', 'Imām', '나무타기캥거루들', '導', '济', '콴뎬', '菊', '얄티랸', '釋', '屈', '坂', '障', '卵', '巢', '閼', '滎', '뮈', '景', '챗필드', '晳', '３초', '價', '카멜라우쿰', 'Éire', '補', '照', '내쫒', '敏', '桃', 'Φαίδρα', '丘', '稚', '褶', '얀챠', '오툉', '萩', '拳', '孔', '시뻘겋', 'コシャマインの', '쨍쨍', '藏', '르퓌', '쿠첵이', 'क़ुली', '陰', '馮', '数', '衛', '單', '丕', '遷', '니콜라옙스크', '借', '姬', '↑↓↑↓식', '劍', '衝', '血', '卓', '菅', '内', '룀', '띨', '일리리쿰', '僑', '圓', 'Große', '痙', '纂', '放', '붐볐', '쥘레', '징그럽다거나뽈록뽈록', '쑨원', '奉', '만듦', '애꿎', 'Δ΄', '珽', '슌스케', '엊갈렸', '８척', '欧', '覺', '기젱', '괘씸죄', '잴', '헹가래', '具', '牛', '아퀼라', '飛', '퓌스텔', '砲', '病', '計', '佃', '켐니츠', 'Stanisław', '썻', '아뎀', '弧', '툇마루', '郵', '呼', '震', '이퀩먼트', '邊', '令', '사이드１', '홋카이도청', '퍄샤', 'サンヨーオールスターゲーム', '泉', '壱', '奬', '트레믈레', '남겨젔', '앎', '器', 'Þóris', '牲', '테르모퓔라이', '稹', '允', '渭', '鼎', 'ᛚᚢᚴᛁ', '恩', 'ㅚ', '荘', '뎦혀', '顯', '軟', '好', '術', '匡', '號', '섐플레인호', '汪', '뫼욱스', '禪', '蜈', '希', 'κατάφρακτος', '農', '妖', 'アダルトチルドレン', '볏짚', '뢰미쉔', '魚', '造', '沼', '쳇데기', '停', '賞', '뮤코펩티드', '領', '駅', '兢', '浅', '璣', 'المعتزلة', '알류샨', '副', '湖', '昆', '돝', '휜카르', '襲', '宣', '우솝', '濃', '굄돌', '켐브리지', '내리쬔', '慶', '台', '鬹', '璧', '禍', '歸', '半', '阮', 'メカドン', '標', '竝', '甚', '鏡', 'সেতার', '目', '스폿트실베니', '斑', '倒', '티스콴텀', '햣겐', '烹', 'НІ', '생오벵쉬르메르', '淵', '笹', '植', '괵공', '徐', '胤', '튤립', '윔블', '督', 'めづる', '돤치루이', '두샨베', '付', '彦', '피웁', '루카셴코', '銖', '辺', '클롼', 'ノ', '縣', '範', '트리뷴', 'княз', '뱃삮', 'Μήδεια', '參', '李', '갉아', '靜', '밈', '訕', '쳇바퀴', '푝력', '웻랜즈', '割', '헀', 'מׇשִׁחַ', '홋카이', '리얼소꿉놀이', '陶', '腺', '리츨', '¹', '菩', '蒙', '젱킨스', '駐', '燈', '辯', '터븀', '쭌', '因', '製', '兴', 'Säpo', '吳', '根', '炯', '刺', 'كربغا', '캥커', '帥', '儼', '齋', '홋카이도제국대학', '拱', '샨시', '훙레', '이큅먼트', '雅', '웸블리사', '祚', '傅', '斡', '두샨', 'повешенных', '焼', '盧', '眼', '앳킨슨', '쑹타오', '苾', '콰르텟', '숄덴', '移', '尖', 'µH', '楽', '離', '카르보니움', '詞', '略', 'vǫlva', '札', '始', '向', '宋', '豫', '牙', '斗', '吏', '局', '扶', '草', '땋', '하나쨩', '幢', '貴', '멥쌀', '퇼처', '裏', '사마휼에게', '촐라푸람이', '壓', '카렝탕', '環', '麗', '托', '橋', '아버스놋', '꿴', '包', '묄커바슈타이', '스뮈츠', '區', '괵숙', '鉱', 'Народничество', '妃', '．', '提', '쿠챤', '。', '灏', '説', '咬', '樣', '訟', '兼', '千', '取', '벵골', '듄', '나뉠', '쾰른토나토', '쿠샨', '신뇽', '몬시뇰', '辣', '폴묄러', '釀', 'Eiríkr', '映', '批', '極', '超', '겜린', '드레퓌스', '비츨레벤', '倉', '勅', '凸', '綱', '墳', '욋가지', '햘티', 'Μεγάλος', '젱긴', 'あびこ', '퀸텟', '宰', '筋', '徒', '錯', '쥘', '隨', '뜷린', '牌', '헤룔프손', '乾', '擴', '炙', '坐', '베이쭝부', '媒', '劇', '비욘드', '姦', '柳', '괵국부인', 'الكاشغري', '⑧', '午', '戶', '젭', 'wählen', 'Władysław', '償', '暹', '곯아', '수탉', '程', '凱', '숀', '추옝', '題', '陂', '額', '洗', '찝찝', 'מׇשִׁיחַ', '音', '벨렝탑', '態', '뙤약볕', '憐', '昭', '鼻', '홧김', '뤽상부르', '챕터', 'Gräbner', '廙', '뉴브런즈윅', '沖', '뷜레이스트', '誘', '깨부', '圍', '赤', '짹', '葬', '竿', 'Bình', '솔레놉신', '린뱌오', '増', '宿', 'Василия', '服', '카르복시펩티다아제', '曲', 'Arbëreshë', '忍', '撫', '懿', '岩', 'hÉireann', '列', '豊', '痛', '부서젔', '衡', '銓', 'Óenfhir', 'Biêng', '∧', '⇌', 'Rơwoa', '裙', 'Þorgnýrson', '茶', '헝겊', '괭이갈매기', '아퀼', 'ㅟ', 'スルトキハ', '熙', 'アイル', '戸', '叙', '樹', '사이드３', '弐', '鎭', '肉', '嶺', '凶', '賤', 'ロシアパン', '베이욘', '蘇', '驛', '설하멱', '寬', '니콜라옙스크', '創', '달궜', '粉', '厚', '梨', '鹿', 'Jagiełło', '빻', '類', '峴', '察', '＼', '船', '본슌닛키', '惟', '龜', '⑨', '伍', '菑', '旨', '鄕', '按', 'Fidèle', 'Trần', '다퉜', '죗값', '綿', '頂', '쪼갰', '匠', '毅', 'β', '奇', 'Borgarhjǫrtr', '袁', '에스파뇰라', '密', '스웜프', '圭', '澤', '棋', '犬', '薩', '護', '햣켄', '尊', 'Πάτρα', '죈', '코뮨주', '푸깟현', '날쌔', '指', '虎', '거셌', '朱', '郳', '項', '量', '힘썻', 'казакі', 'Αθηναι', '깰', '弁', '懷', '프로옉투스', '権', 'Đại', '崖', '兪', '秦', '純', '洑', '尼', '竪', '슾지', '右', '冠', '昌', '童', '彫', '徽', 'каза́ки', '겯', '弘', '擬', '훙가로톤', '楊', '即', '진줏빛', 'α', '班', '놋쇠', '陀', 'Hænsa', '泓', '毫', '귈리엘모', '인캇츠', '도롱뇽', '總', '소꿉친구였', '頭', '妻', '灣', '폴리펩타이드', '難', 'Лев', '鉄', '旭', '宇', '뼛조각', '塑', '毒', '迎', '箕', 'Torrejón', '泳', '儀', '屍', '奭', '妥', '済', '脫', '遼', '算', '里', '배곯', '井', '000켤레', '艦', '驍', '郎', '돛', '앵귈라', '스훙토', '隆', '寵', '卞', '置', '蒼', '勢', '劣', 'Liêng', '루씰', 'Mandić', '腐', '嗣', 'モダン', '홋카이도척식은행', '両', '경햐', '季', 'メカカツ', '願', '揚', '犁', '発', '보괸되', '滷', '모뎀', '坮', '嫗', '琵', '漕', '스렝', '크리스챤', '歐', '兒', '헵번', 'インドン', '巴', '園', '扈', '鑑', '견뎠', '筒', '仇', 'ætheling', '委', '영햑력', '良', 'Hansdóttir', '丹', '濤', '漁', '薰', '崔', '랸', '３일간', '깼', '蔡', '祭', '니오븀', '胴', '부챗살', '港', '区', '芝', '票', '1㎠마', '홋카이도', '퓌스콘', '荒', '輝', '惠', 'Stærkodder', '鶴', '鳳', 'Στύγα', '페뇽데벨레스데라고메라', '셧다운', '셍글레', '하탸야', '剝', '신욧카이치', '돤치루', '杞', '깽깽', '甘', '쾨펜', '陵', '弱', '沛', '沈', '釉', '瓜', 'Рассказ', '塗', '다이샨과', '澎', '옅', '흄', '켐페인', '封', '베들레헴', '燒', '丞', '藩', '雑', '角', '３회', '諶', '사귐섬', '스콴토', '雄', '푀데라티', '껐', '아델리펭귄', '하곘', '屬', '臺', '？', '壺', '請', '꼰강', 'Leó', '韋', '쑹장', '시킵', '步', '櫃', '珠', '베르윅', '발언헀', '驊', '孚', '뮈스텔', '稿', '섄탈', 'ヶ', '철툇자루', 'सितार', '永', '総', '느바욧', '禁', '僚', '수퇘지', '부르귀뇽', '宏', '갭', '悛', '건넜', '齊', '被', 'φαιδρός', '鼓', '巖', '그뢴', '猪', '６월', '爾', '가뇽', '뫼비우스', '妊', '洞', '슈펭글러', '헹세', '種', '阳', '炭', '籍', '魂', '횃불', '皮', '穆', '尙', 'infidèles', '恭', '伴', '펭귄', '何', '퓨젼', '투르넴', '格', '葯', 'Musiał', '繁', '貫', '黒', '獸', 'Sævarstöð', '샨킬', 'panetùn', '자뱌드', '需', '세지윅', '헵번', '尾', '뱌쿠렌', '敦', '이솝우화', '仲', '弼', '振', '트롱트낵', '晟', '타임갭', '숴레이', '杜', '鉉', '述', '捨', '稷', '巷', '望', '型', '뎃포', '가오슝', '袍', '玄', '姫', '工', '疊', '丑', '稅', '의미햐', '–', '테뫼식', '卯', '対', '慰', '策', '塼', '뱌르니', '侗', '彪', '譚', '맬컴', '彩', '雨', '큭', '釜', '追', '로엡', '哉', '鳴', '遺', '疑', '岳', '쾍', '講', 'تار', '찧', '征', 'Hveðrungr', 'ˈeːrʲu', '촨차', '御', 'だが', '盞', '侯', '슌', '背', '枝', '쟀', '톳', '珏', '系', '동여맸', '鷄', '聚', '夷', '뜷', '敵', '벰보', '蟹', 'Ю́рий', '忠', '託', 'Aífe', '胎', '賢', '챕터이', '离', '骨', 'ありなしコンビ', 'カ', '漫', '譜', '楚', '壘', '凹', 'トラウマサバイバー', '謝', '荷', '依', '뺵', '住', 'Миха́йлович', '虔', 'チ', '摯', '樑', '廊', '紅', '소꿉', 'تقیة', '뮈소', '쉔톤', '訶', '店', '5±1', '棄', '穫', '姉', '松', '詔', '부숴', '俘', '附', '階', '添', '喪', '舊', '鄲', '耒', '申', '姚', '토르놉스키', '譲', '細', 'メビウス', '尺', '에스파뇰', '멜키세첵', '管', '댜틀로프', '娘', 'mù', '果', '宜', '품삯', '시헙', '코눙그', 'て', '부됸니', '틔', '여덞', '豪', '闕', '邱', '유셴코', '킵챠크', '消', '烈', 'Ἰώ', '頡', '鄴', '웽거', '等', 'Στύξ', '液', '힛위켓', '크루섕크', '賴', '紙', 'œil기법', '繫', '夔', '頼', '긱스', '프롱트낵', '藤', '쫒기', '페뇽데알우세마스', '５척', '衣', '쫒겨났', '쳄발로', 'オランピア', '일궜', '아비뇽', '邯', '骅', '굿펠로우나무타기캥거루', '麵', '카드뮴', '勞', '丸', '郡', '瑾', '帽', '弔', '緖', '更', '趙', '体', '粹', '着', '쿠샨계', '불스갭', '브런스윅', '시오야슌', '스웩', '郭', '자오쩬민', '俊', '씰', '屋', '益', '苦', '敍', '°C', '마욘', '閣', '迷', '디켐버', '監', 'Þröstur', '簡', '鍵', '屯', '函', 'ロマン', '砂', '테르모퓔라이', '豹', '輕', '壁', '캥거루류', '盟', '겅솽', '앳', '脂', '힝기스', '아뢨', '왝시', '宸', '穰', 'سی', '촐라푸람', '帖', '곪', '坊', '킵차크', '諭', 'ón', '錦', '胥', '材', '琴', 'ستار', '実', '아폽토시스', 'Þóra', '智', '梅', '蜎', '헹가레', '브런즈윅', '滿', '猶', '菜', 'الله', '젋', '少', '세묜', '祿', '비욘드', '團', '交', '祥', '潭', 'المملوك', '廢', '侍', '魯', '감성적ᐨ에로스적', '簪', '綬', '電', '지저귐', '墓', 'சோழ', '蟲', '絢', '꿸', '괵국부', '카르마뇰라', '板', '覓', '底', '하얏트', '財', '벱스', '푸르트벵글러', '椅', 'を', '転', '姓', '遇', '蕃', '螭', '涵', '靡', '線', '位', '詹', '신라빠꼰', '座', '從', '波', '粧', 'ささごトンネル', '탯줄', '覆', '퀼리예라', '엊도', '伯', '핼러데이', '跡', '피엘스트룀', '떄', '엡', '륄리', '閤', '¾', '毛', '琿', '贖', '核', '示', '疹', '暗', '胸', '冬', '蓮', '整', '莞', '플랑타뷜리', '니촐라', '乱', '町', '멱살잡이', '速', '야쿱', '杯', '油', '巾', '鱗', '뗀석기', '콱', '巳', 'козаки́', '톳', '殺', '呉', '雍', '망나뇽', '셀렝가강', '惕', '兌', '辞', '怒', '陸', '皇', '均', '齡', '孟', 'クシェル', '스튀르뵤른', 'ヲコト', '梁', '訓', '窓', '軸', '科', '膺', '캉셩', '餘', 'línea', '홀뮴', '蔣', '慣', '粘', '覇', 'Värmland', '玉', '爭', '滌', '活', '衍', '槐', '潔', '舵', '夜', '柹', '메카캇', '黶', '塘', '范', '換', '磨', '万', '믈라카', '땀디엡산', 'ㅕ러', '猇', '匣', '쨍그렁', '候', '켐벨', '樽', '祠', '라우셴버그', '痘', '苑', '克', '斷', 'الإمام', '舍', '爵', '먀스코프스키', '勇', 'シ', '奕', '洸', '碗', '브뢸', '꽹과리', 'Mágach', '뵐', '錫', '句', '芸', '防', '乘', '深', '存', '夢', '에미넴', '褚', '柔', '샤오펀훙', '≪북ᐨ남', '警', '마라뇬', '옻칠', 'のようにならないようにしよう', '備', '回', '拉', '홋스퍼', '없앴', '織', '段', '設', '竹', '諫', '幾', '綠', '採', '벵츠도터', 'ციხე', '刀', '분슌', '桀', '乃', 'かつ', '撞', '洪', '이퀍먼트', '舫', '舌', '錚', '掾', '斐', '廃', '僧', '스케쥴', '스텔쾨더', '寔', '위칳', '曹', '鬲', '⑪', '遮', 'Darraðarljóð', 'Εὔφορβος', '謁', '쳄발', '貳', '邑', '羽', '隱', '后', '還', '尻', '蜀', '賣', '枾', '苔', '縮', '킁킁', '溪', '예멜랸', '穴', '膝', '숄츠', '絶', '筆', '今', '엷', '캥거루', '켤레', '빳빳', '烏', 'Kürboğa', 'Forêts', '왐파노아그', '著', 'Смирно́в', '쾰른안데어슈프레', '뮐러', '劉', '飮', '뮈라', '柱', '彰', '５월', '広', '技', '崇', '必', '翁', '最', '画', '校', '浿', '臨', '핼', '虫', '璞', '考', '괭이갈매기', '布', '畝', '俗', '讀', '알류쟌', '低', '津', '溝', '秀', 'Friðriksdóttir', 'mamă', '興', '条', '傍', '漳', '틍징', '욘', '옻나무', '閔', '變', '苗', '旌', '協', 'ＲＸ', '率', '쇳물', '室', '펨토', '錢', '矣', '舟', '聯', '淪', '다이샨', '핀챠스', '施', '遠', '텄', '熱', '守', 'トンネル', '括', 'семи', '라우쉔버그', '戌', '섶', '琅', '뮈', 'Þorgnýr', '잰', '殷', 'ملك', '飾', '亡', '俣', '痕', '킵초', '隊', '訴', '穀', 'გონიოს', '叔', '秋', '、', '벰', '）', '助', '소꿉놀이', '駿', '걲', '持', '勝', '뵤른', '盛', '槽', '收', '鐵', '琔', '３함대', '縫', '메카캇츠', '加', '璵', '아이튠스', '포르트솁첸코', '狼', '狀', '樂', '어쨋든', '轉', '２월', '脈', '側', '가엾', '洋', '陳', '伸', '象', '仗', '쳇', '讓', '묽기', '베르됭', '牽', '쾨세다그', '櫜', '젭중', '孝', '艇', '그믐날', '抗', '猿', '리위안훙', '젭', '梯', '彈', 'い', '聲', '鈴', '禹', '尹', '肩', '矯', '畿', '哲', '순옻칠', '諸', '偉', '抛', '崩', '루벵', '休', '맬', '進', '攻', '序', 'ジャファル', '珥', '秉', '幸', '控', '国', '硅', '居', '咎', 'maˈkɛdɔnski', '帶', '펩타이드', '假', '鎖', '革', '弟', '짰', '衆', '아퀼리페르', '다롄', '蘿', '盒', '勳', '給', '改', '촐', 'り', '盈', '燕', '할스트룀', '헤맸', '홀뮴', '쇤베르크', '丽', '許', '勒', '笠', '倍', '辰', '啓', '피죤투', '續', '完', '房', '感', 'Hα선', '蟠', '생섵하', '도스토옙스키', '複', '킵', '庸', 'Μιχαήλ', '路', '酒', '短', 'Κένταυροι', '廟', '倫', '햐쿠만고쿠', '捕', '쥰이', '사마휼', '群', '이셴', '핼리팩스', '괭이', '茨', '쐬서', '반짇고리나', '카챠', '올륌퓌아', '囊', '描', '롄잔', '乙', '党', '享', '갬블러', '潢', '博', '翼', 'κατάφρακτοι', '卜', '初', '휨', '泥', '蔕', '泰', '熊', '像', '豆', '蹟', '트레뻬제', '천수이볜', '建', '뵤른손', '제１연합', '再', '뜀틀', '浮', '騎', '♣', 'δρακων', 'εὔδοξος', '퓌헤레노르', '漸', 'めんち', '4～5단', '弓', '웁살라', '企', '貸', '紀', '達', '투하쳅스키', '濁', '노보니콜라옙스크', '奎', 'ヲ', '巫', '幀', '땔', '스타트리뷴', '殿', '評', '欽', 'États', '魏', '藥', '나쁩', '兆', '꺠달', 'ＭＳ', 'ヒ', '쇼아죌', '癸', '오라녜당', '摩', '홑처마', '友', '줏키', '버텼', '욘타마리', '１회', '溫', '다루몐', '蚣', '뱌체슬라프', '羊', '租', 'torrejón', '툇간', '敗', '彌', '노벰버', 'Лира', '棟', '麻', '잽싸', '갭슐', '降', '審', '翅', '岐', '쉘터', '멎', '坡', '踏', '瀧', '함꼐', '越', '端', '醋', '奥', '酸', '淑', '스킵', '盤', '亭', '丙', '姥', '슌센', '가냘픈', '喜', '桐', '宅', '姜', '헤르츨', '浦', '汗']

In [39]:
new_tokens = ' '.join(new_tokens)

In [41]:
new_tokens

'völva 韜 貢 顧 嶽 （ 웡원하오 淳 حزب 儒 紂 쫒 ὁπλίτης ～ 喬 鳥 샥티파 兀 叉 洲 위스트레헴 캇츠 佩 碑 逆 危 议 壽 햐쿠하치 贛 賀 薄 핼시 à 켤 뗏목 素 虞 爺 Fesztiválzenekar 堀 旣 延 谷 胡 압뒬카디르 流 沙 鍾 웸블리 派 퀀틱 ク 꿰맸 壬 떤선녓 刹 塚 畸 索 誤 호엔촐레른 球 Пак 詠 Λιμός 뤽상부르궁 지샜 架 誌 紋 継 땝납 識 욧카이치 庚 槪 팼 送 Moái 퀘벡ᐨ파리문학상 蠻 誹 叭 趺 彝 龕 層 E° 妙 고르노바다흐샨 삥삥 번쨰 牟 樞 鏞 홑몸 饗 浪 隠 힉먼 耆 否 امام 윅 排 Cúscraid 蓋 Elín 紹 சாம்ராஜ்யம் Ｖ 学 結 アッカーマン 잽 普 듄 thiên 癲 Chèvremont 庵 偃 꾸밈 히믈러 航 禑 횃불 幹 ３ 槍 亥 閭 廳 清 隋 館 符 务 視 矢 優 韻 免 돛대 揮 옝기사르 輪 炎 띌 薛 어댑터 律 納 캇페 奴 샥터 ← 起 휜 넙다리뼈 壹 ± 他 뮐베르크 香 mulūk 먀오디거우 臂 謀 癎 Φάλαγγα 星 疾 泛 황즈슝 專 腱 lǫgmaðr 베렝가르 止 恋 浩 제라프샨강 み 郷 恕 杖 노보셀렝긴스크 羅 除 아델리펭귄 뷜로 に 役 wēnlŭmiàn 羨 墻 Thiêm 貊 旻 켁 買 쵀디 샨시성 途 창립멥버 ㅎ뢀약 渲 ˈjazik 参 부디놋 丈 し 競 伺 歲 旗 網 街 貞 벵골만 週 테셴 닻 池 救 舞 嬪 粒 灝 次 六 巡 黎 카탸 塽 雁 đường 域 印 承 퓌르논 淞 肥 殻 璽 敬 執 糞 쾰른 贊 킵초케 갬블 猛 雙 呂 味 羌 磁 minīn 큭큭 堵 談 葉 特 湜 寶 鬼 瑞 直 夏 順 색옻칠 壇 禅 쇤스키 品 耽 件 健 毘 柄 青 老 묽 戦 뷀츠부르크 構 左 慾 야르믜슈 惣 쾰른 霊 調 프리츨라어 챕터나 營 슬펐 留 야쿱 劑 厲 폽풀라레스 犯 恂 醫 仙 歷 여읜 腹 맟추 어븀 鑊 鹽 聞 急 省 狂 ならはしみき 析 湯 症 點 就 Παφλαγών 옻칠기 首 François 竜 Amīr 增 소꿉친구 潘 瓊 雪 糖 嬴 奏 도롱뇽목 衒 娠 დავით 쓰촨 罪 쿠첵 瀛 쫒아내

In [43]:
import re 
s='岐 쉘터 멎 坡 踏 瀧 함꼐 越'
hangul = re.compile('[^ ㄱ-ㅣ가-힣+]') 
# 한글과 띄어쓰기를 제외한 모든 글자 # 
hangul = re.compile('[^ \u3131-\u3163\uac00-\ud7a3]+') 
result = hangul.sub('', s) 



print(result)

 쉘터 멎    함꼐 


In [49]:
hangul = re.compile('[^ \u3131-\u3163\uac00-\ud7a3]+') 
result = hangul.sub('', new_tokens)

In [50]:
result = result.split()

In [52]:
print(result)

['웡원하오', '쫒', '샥티파', '위스트레헴', '캇츠', '햐쿠하치', '핼시', '켤', '뗏목', '압뒬카디르', '웸블리', '퀀틱', '꿰맸', '떤선녓', '호엔촐레른', '뤽상부르궁', '지샜', '땝납', '욧카이치', '팼', '퀘벡파리문학상', '고르노바다흐샨', '삥삥', '번쨰', '홑몸', '힉먼', '윅', '잽', '듄', '꾸밈', '히믈러', '횃불', '돛대', '옝기사르', '띌', '어댑터', '캇페', '샥터', '휜', '넙다리뼈', '뮐베르크', '먀오디거우', '황즈슝', '베렝가르', '제라프샨강', '노보셀렝긴스크', '아델리펭귄', '뷜로', '켁', '쵀디', '샨시성', '창립멥버', 'ㅎ뢀약', '부디놋', '벵골만', '테셴', '닻', '카탸', '퓌르논', '쾰른', '킵초케', '갬블', '큭큭', '색옻칠', '쇤스키', '묽', '뷀츠부르크', '야르믜슈', '쾰른', '프리츨라어', '챕터나', '슬펐', '야쿱', '폽풀라레스', '여읜', '맟추', '어븀', '옻칠기', '소꿉친구', '도롱뇽목', '쓰촨', '쿠첵', '쫒아내', '게임스폿', '라오읻싸', '와치랄롱꼰', '아퀼레이', '윔블던', '켐페', '나눗셈값', '아녜스', '세', '묄렌도르프', '포르텡베생', '폴랸스키', '모켕탕동', '분', '도룡뇽', '유로퓸', '티모셴코', '위웡섬', '퓔레', '맬콤', '라오읻싸라', '킬로', '뮐', '촐라', '브륀힐드', '남캅카스', '뗀', '스퀠쾨더', '땔감', '나뮈르', '촐몬데레', '무깟디마', '틔웠', '핥', '이스파냑', '보살핌', '시퀀스', '브런즈윅', '뭍', '쿨부샨', '대', '햅번', '캇파투카', '테르모퓔라', '촐라국', '셍캉', '뫼즈', '암탉', '비숑', '캥거루섬', '윔베르', '쑹화', '스텟', '쓰촨성', '겻', '이리듐', '굄목', '쿰셍기르', '메나헴', '나무타기캥거루들', '콴뎬', '

In [36]:
import re
# new_tokens = re.sub('[ぁ-ゔ]+|[ァ-ヴー]+[々〆〤]','', new_tokens)
new_tokens = re.sub('一-龥','', new_tokens)

In [31]:
new_tokens = re.compile('[가-힣] + ').findall(new_tokens)

In [37]:
new_tokens

'völva 韜 貢 顧 嶽 （ 웡원하오 淳 حزب 儒 紂 쫒 ὁπλίτης ～ 喬 鳥 샥티파 兀 叉 洲 위스트레헴 캇츠 佩 碑 逆 危 议 壽 햐쿠하치 贛 賀 薄 핼시 à 켤 뗏목 素 虞 爺 Fesztiválzenekar 堀 旣 延 谷 胡 압뒬카디르 流 沙 鍾 웸블리 派 퀀틱 ク 꿰맸 壬 떤선녓 刹 塚 畸 索 誤 호엔촐레른 球 Пак 詠 Λιμός 뤽상부르궁 지샜 架 誌 紋 継 땝납 識 욧카이치 庚 槪 팼 送 Moái 퀘벡ᐨ파리문학상 蠻 誹 叭 趺 彝 龕 層 E° 妙 고르노바다흐샨 삥삥 번쨰 牟 樞 鏞 홑몸 饗 浪 隠 힉먼 耆 否 امام 윅 排 Cúscraid 蓋 Elín 紹 சாம்ராஜ்யம் Ｖ 学 結 アッカーマン 잽 普 듄 thiên 癲 Chèvremont 庵 偃 꾸밈 히믈러 航 禑 횃불 幹 ３ 槍 亥 閭 廳 清 隋 館 符 务 視 矢 優 韻 免 돛대 揮 옝기사르 輪 炎 띌 薛 어댑터 律 納 캇페 奴 샥터 ← 起 휜 넙다리뼈 壹 ± 他 뮐베르크 香 mulūk 먀오디거우 臂 謀 癎 Φάλαγγα 星 疾 泛 황즈슝 專 腱 lǫgmaðr 베렝가르 止 恋 浩 제라프샨강 み 郷 恕 杖 노보셀렝긴스크 羅 除 아델리펭귄 뷜로 に 役 wēnlŭmiàn 羨 墻 Thiêm 貊 旻 켁 買 쵀디 샨시성 途 창립멥버 ㅎ뢀약 渲 ˈjazik 参 부디놋 丈 し 競 伺 歲 旗 網 街 貞 벵골만 週 테셴 닻 池 救 舞 嬪 粒 灝 次 六 巡 黎 카탸 塽 雁 đường 域 印 承 퓌르논 淞 肥 殻 璽 敬 執 糞 쾰른 贊 킵초케 갬블 猛 雙 呂 味 羌 磁 minīn 큭큭 堵 談 葉 特 湜 寶 鬼 瑞 直 夏 順 색옻칠 壇 禅 쇤스키 品 耽 件 健 毘 柄 青 老 묽 戦 뷀츠부르크 構 左 慾 야르믜슈 惣 쾰른 霊 調 프리츨라어 챕터나 營 슬펐 留 야쿱 劑 厲 폽풀라레스 犯 恂 醫 仙 歷 여읜 腹 맟추 어븀 鑊 鹽 聞 急 省 狂 ならはしみき 析 湯 症 點 就 Παφλαγών 옻칠기 首 François 竜 Amīr 增 소꿉친구 潘 瓊 雪 糖 嬴 奏 도롱뇽목 衒 娠 დავით 쓰촨 罪 쿠첵 瀛 쫒아내

In [40]:
new_tokens = list(set(new_tokens))

In [41]:
len(new_tokens)

2220

In [42]:
vocabulary = list(tokenizer.vocab.keys())
for token in new_tokens  :
    if token in vocabulary :
        print('token')

In [43]:
num_added_toks = tokenizer.add_tokens(new_tokens)
model.resize_token_embeddings(len(tokenizer))

Embedding(34220, 1024)

In [44]:
print(tokenizer.tokenize('\\n'))

['\\n']


In [45]:
from transformers import AutoModel, AutoTokenizer

model = AutoModel.from_pretrained("klue/roberta-large")
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-large")

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it f

In [46]:
tokenizer.add_tokens(new_tokens)
model.resize_token_embeddings(len(tokenizer))

Embedding(34220, 1024)